# Parameters fitting

In [ ]:
%matplotlib inline
from xmgrace_fit import nlfit, wsrc_nlfit
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import *

from scipy.stats import chisquare, linregress
from pathlib import Path
import math
from lmfit import Model
import pandas as pd
import seaborn as sns

sns.set_style('ticks') # setting style
sns.set_context('paper') # setting context
sns.set_palette('colorblind') # setting palette


## 0. Utils

In [ ]:
DEST_PATH = Path.cwd() / Path("outputs")
DATA_PATH = Path.cwd() / Path("data")
XMGRACE_INPUT_PATH = Path("xmgrace_inputs") # This is a temporary folder that stores xmgrace input dat files generated by save_data function
EXT_FILE = ["eps"]
COLORS = sns.color_palette("colorblind")
COLOR_ORANGE = COLORS[1]
COLOR_GREEN = COLORS[2]
COLOR_BLUE = COLORS[0]

In [ ]:
def load_data(pathname):
    """
    Load data from a file.
    pathname: str, path to the file
    """
    data = np.loadtxt(pathname, unpack=True, comments="#")
    return data

def save_data(filename, header, cols):
    """
    Save data to a (dat) file.
    """
    output = np.column_stack(cols)
    np.savetxt(filename, output, fmt='%10.5f', delimiter='\t', header=header)

## II. WSRC fitting

### 1. ADC conversion of the ESP32 board 
> used for the measurement of the WSRC output voltage

In [ ]:
filename_calib_carte = "calib_carte_vair.dat"
data_calib_carte = load_data(DATA_PATH / filename_calib_carte)

ref_voltage = np.asarray(data_calib_carte[0])
esp32_voltage = np.asarray(data_calib_carte[1])

res = linregress(esp32_voltage, ref_voltage)

# y = a_coeff * x + b_coeff
a_coeff = res.slope
b_coeff = res.intercept
print("y = {:1.3f} * x + {:1.3f}".format(a_coeff, b_coeff))
print(res.rvalue**2)
print(f'slope {res.stderr}')
print(f'intercept {res.intercept_stderr}')
fig, ax = plt.subplots()

ax.scatter(esp32_voltage, ref_voltage, marker='+')
ax.plot(esp32_voltage, a_coeff * esp32_voltage + b_coeff)

ax.minorticks_on()
ax.tick_params(axis='both',labelsize=12)
ax.set_xlabel('Digital value from ADC (12 bits)')
ax.set_ylabel('Ref. Voltage $(V)$')
ax.grid(True,'major','both')

props = dict(boxstyle='round', facecolor='white', alpha=0.7)
ax.text(0.5, 0.3,"y = ${:1.2e}\cdot x + {:1.2e}$\n$R^2$ = {:1.3f}".format(a_coeff, b_coeff, res.rvalue**2),  transform=ax.transAxes,  verticalalignment='top', bbox=props)

plt.show()

def f_uv_esp32(u_esp32):
    return a_coeff * u_esp32 + b_coeff


### 2. WSRC data reading

In [ ]:
filename_300323_1 = "Air_300323-1_comfortsensor001_Vcc5V.dat"
data_air = load_data(DATA_PATH / filename_300323_1)

In [ ]:
MIN_OFFSET = 0
MAX_OFFSET = 19

# U SENSOR
u_volt = np.asarray(data_air[4])[MIN_OFFSET:MAX_OFFSET]
err_uvolt = np.asanyarray(data_air[5])[MIN_OFFSET:MAX_OFFSET]

# TESTO
vair = np.asarray(data_air[1])[MIN_OFFSET:MAX_OFFSET]
err_vair = np.asarray(data_air[2])[MIN_OFFSET:MAX_OFFSET]
temp_air = np.asarray(data_air[3])[MIN_OFFSET:MAX_OFFSET]#T testo

RVPin_sensor = np.asarray(data_air[6])[MIN_OFFSET:MAX_OFFSET]
delta_RVPin_sensor = np.asarray(data_air[7])[MIN_OFFSET:MAX_OFFSET]

### 3. Plot U = f(Wind speed)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(vair,u_volt, marker='+')
ax.minorticks_on()
ax.tick_params(axis='both',labelsize=12)
ax.set_xlabel('Wind speed - Ref. $(m.s^{-1})$')
ax.set_ylabel('$U_{out,dg} (V)$')
ax.grid(True,'major','both')
plt.show()

### 2. WSRC fitting relativly the King's law function

King's law :

$$U^2 = R_w(T_w-T_{air})(a+b \cdot v_{vair}^n)$$

In [ ]:
RW = 10E3
def king_s_law(X, tw, a, b, n):
    '''
    x : vair
    Parameters to fit :
    - TW (constraint : TW > 30°C)
    - a
    - b
    - n
    Ref : Cimbala and W.J. Park /  Khamshah 2011 10.5897/IJPS11.630  
    '''
    tempair, vair = X
    return RW*(tw - tempair)*(a + b*vair**n)

Preprocessiong of data 

* $x = vair$ (Wind speed)
* $y = f_{uv, exp 32} (rv_{pin})^2$ 

In [ ]:
x_esp32 = vair # Vair measured with the reference
y_esp32 = (f_uv_esp32(RVPin_sensor ))**2 # Data measured with ModernDevice => *5/4095 conversion in Volts
deltay_esp32 =  (f_uv_esp32(delta_RVPin_sensor))**2 #Error on Modern Device Data => *5/4095 conversion in Volts

In [ ]:
x_mult = vair
y_mult = u_volt**2
deltay_mult = err_uvolt**2
y_mult

In [ ]:
m = Model(king_s_law)
params = m.make_params(tw=dict(value=114.662, vary=False),a=1.,b=1.,n=0.4, method='leastsq')
fit_air_mult = m.fit(y_mult,params, X=(temp_air, x_mult))
print(fit_air_mult.fit_report())
print(fit_air_mult.params.pretty_print())

In [ ]:
m = Model(king_s_law)
params = m.make_params(tw=dict(value=79.527, vary=False),a=1.,b=1.,n=0.4, method='leastsq')
fit_air_esp32 = m.fit(y_esp32,params, X=(temp_air, x_esp32))
print(fit_air_esp32.fit_report())

In [ ]:
fig, ax = plt.subplots()


ax.plot(x_esp32, fit_air_esp32.best_fit, label = '$U_{fit,esp32}^2$', color=COLOR_BLUE)
ax.plot(x_esp32,y_esp32,'+', label = '$U_{out,esp32}^2$')
ax.plot(x_mult, fit_air_mult.best_fit, '--', label = '$U_{fit,dg}^2$', color=COLOR_ORANGE)
ax.plot(x_mult, y_mult, 'x', label = '$U_{out,dg}^2$')
ax.tick_params(axis='both')
ax.set_xlabel('Wind speed - Ref. $(m.s^{-1})$')
ax.set_ylabel('$U^2$ $(V^2)$')
ax.legend()
ax.minorticks_on()
ax.grid(True,'major','both')

props = dict(boxstyle='round', facecolor='white', alpha=0.7)

for ext in EXT_FILE:
    plt.savefig(DEST_PATH / Path(f"WSRC_fit.{ext}"))

### Wind speed calibration law
Fitted paramaters : $T_W, a, b, n$

$$v_{air}(U) = \sqrt[n]{\frac{1}{b} \frac{U^2}{R_W (T_W - T_{air})}}$$

In [ ]:
RW = 10E3
TW = 79.5
A = 5.693E-6 
B = 3.17E-6
N = 0.44
def vair(U, tair, rw=RW, tw=TW, a=A, b=B, n=N):
    return ((1/b)*((U**2)/(rw * (tw - tair))-a))**(1/n)

## III. MRT sensor fitting

### 1. MRT data reading

In [ ]:
filename_mrt = "MRT_061023-091023.dat"
data_mrt = load_data(DATA_PATH / filename_mrt)
T_testo = np.asarray(data_mrt[0])
T_DS18B20 = np.asarray(data_mrt[1])


### 2. MRT fitting

In [ ]:
res = linregress(T_DS18B20, T_testo)

# y = a_coeff * x + b_coeff
a_coeff = res.slope
b_coeff = res.intercept
print("y = {:1.3f} * x + {:1.3f}".format(a_coeff, b_coeff))
print(res.rvalue**2)
fig, ax = plt.subplots()
ax.scatter(T_DS18B20, T_testo, marker='+')
ax.plot(T_DS18B20, a_coeff * T_DS18B20 + b_coeff, label="Linear regression")

ax.minorticks_on()
ax.tick_params(axis='both')
ax.set_xlabel('T DS18B20 $(°C)$')
ax.set_ylabel('T Testo $(°C)$')
ax.grid(True,'major','both')
props = dict(boxstyle='round', facecolor='white', alpha=0.7)
ax.text(0.7,0.5,"y = ${:1.2f}\cdot x {:1.2f}$\n$R^2$={:1.3f}".format(a_coeff, b_coeff, res.rvalue**2),  transform=ax.transAxes,  verticalalignment='top', bbox=props)
for ext in EXT_FILE:
    plt.savefig(DEST_PATH / Path(f"MRT_fit.{ext}"))

print(f'slope {res.stderr}')
print(f'intercept {res.intercept_stderr}')

### 2.2 $\tau$ fitting

In [ ]:
def T_func(t, a, tau, b):
    return a*(1-np.exp(-t/tau)) + b

In [ ]:
for n, f in [(1, "06102023_T1.dat"),   (2, "09102023_T2.dat"), (4, "09102023_T4.dat")]:
    filename = f"to_fit_treponse/{f}"
    data_T = load_data(DATA_PATH / filename)
    t = np.asarray(data_T[0])
    T_TESTO_T = np.asarray(data_T[1])
    T_DS18B20_T = np.asarray(data_T[2])
    #save_data("T_TESTO_" + f, "X \t Y", t, T_TESTO_T)
    try :
        m = Model(T_func)
        params = m.make_params(a=1, tau=1., b=1., method='leastsq')
        fit_TESTO_T = m.fit(T_TESTO_T, params, t=t)
        print(fit_TESTO_T.fit_report())

        m = Model(T_func)
        params = m.make_params(a=1, tau=1., b=1., method='leastsq')
        fit_DS18B20_T = m.fit(T_DS18B20_T, params, t=t)
        print(fit_DS18B20_T.fit_report())


        fig, ax = plt.subplots()
        tau_TESTO = fit_TESTO_T.uvars['tau']
        tau_DS18B20 = fit_DS18B20_T.uvars['tau']
        ax.plot(t, fit_TESTO_T.best_fit, '--', label = fr'fit: T TESTO T{n} ($\tau={tau_TESTO}$)', color=COLOR_ORANGE)
        ax.plot(t, T_TESTO_T,'+', label = f'T TESTO T{n}', markersize=5, alpha=0.4, color=COLOR_ORANGE)
        ax.plot(t, fit_DS18B20_T.best_fit, label = fr'fit: T DS18B20 T{n} ($\tau ={tau_DS18B20}$)', color=COLOR_BLUE)
        ax.plot(t, T_DS18B20_T, 'x', label = f'T DS18B20 T{n}',  markersize=5, alpha=0.4, color=COLOR_BLUE)
        ax.grid(True,'major','both')
        ax.minorticks_on()
        ax.legend()
        ax.set_ylabel('Temperature $(°C)$')
        ax.set_xlabel('Time $(s)$')
        for ext in EXT_FILE:
            plt.savefig(DEST_PATH / Path(f"MRT_Tau_T{n}_fit.{ext}"))
    except :
        continue

## IV. Noise sensor fitting

### 1. $L_{eq}$ models

In [ ]:
def func_noise(x, a, b):
    return 20*np.log10(a*x +b)


def func_noise_linear(x, a):
    return 20*np.log10(a*x)


### 2. Noise sensor data reading

In [ ]:
# Folder storing all the calibration dat: one CSV file per noise level
SOUND_FOLDER = "SOUND/CALIBRATION_21032024/"

# File containaing the name association table between noise levels and file names
filename_noise = SOUND_FOLDER + "sound_sensor_data.dat"
data_noise = load_data(DATA_PATH / filename_noise)
ref_noise_level = np.asarray(data_noise[0])

record_names = ["data/" + SOUND_FOLDER + str(int(d))+'.csv' for d in data_noise[1]]

# Dictionary associating noise levels to file names
records = dict(sorted({key: val for key, val  in zip(ref_noise_level, record_names)}.items()))

# Outliers
del records[95.2]
del records[93.3]


In [ ]:
mean_voltage = list()
p_to_p_amp = list()


p_to_p_amp_avg = list()
p_to_p_amp_err = list()

amplitudes = dict()
avg_delta_t = list()


for k, v in records.items():
    # Iterate over all the records
    data = pd.read_csv(v)
    t0 = data["timestamp"][0]
    timestamp = data["timestamp"] - t0

    uV = np.asarray(data["uV"]) / 1E6
    rv = np.asarray(data["digital"]) * 3.3 /4095
    mean_voltage.append(np.mean(rv))
    WINDOW_SIZE = 10

    
    #############################################################
    # Compute the peak to peak amplitude through a sliding window
    #############################################################
    
    window_size = 10
    step = 10
    window_nbr = len(uV) / window_size
    cursor = 0
    amplitudes[k] = list()
    while cursor + window_size <= len(uV):
        window_uV = rv[cursor: cursor + window_size]
        amplitudes[k].append(max(window_uV) - min(window_uV))
        cursor += step
    
    # Compute the median and the standard deviation of the amplitudes
    p_to_p_amp_avg.append(np.median(amplitudes[k]))
    p_to_p_amp_err.append(np.std(amplitudes[k]))
    
ref_noise_level = np.asarray(list(records.keys()))
print(f"Mean voltage : {np.mean(mean_voltage)}")


In [ ]:
fig, ax = plt.subplots()
p_to_p_amp_avg = np.array(p_to_p_amp_avg)
ax.errorbar(p_to_p_amp_avg, ref_noise_level, xerr=p_to_p_amp_err, fmt='+')
ax.minorticks_on()
ax.tick_params(axis='both',labelsize=12)
ax.set_xlabel('Peak-to-peak amplitude $(V)$', fontsize=12)
ax.set_ylabel('$Ref level (dB)$', fontsize=12)
ax.grid(True,'major','both')
ax.legend()
plt.show()

In [ ]:
save_data("noise-ptp.dat", "X \t Y", [p_to_p_amp_avg, ref_noise_level])
a, b = nlfit(XMGRACE_INPUT_PATH / "noise-ptp.dat", "20*log10(A0*x + A1)", print_output=True, a0=1, a1=1).values()

In [ ]:
m = Model(func_noise)
params = m.make_params(a=a,b=b, method='leastsq')
fit_noise = m.fit(ref_noise_level,params, x=p_to_p_amp_avg)
print(fit_noise.fit_report())
print(fit_noise.params.pretty_print())

In [ ]:
save_data(XMGRACE_INPUT_PATH / "noise-ptp.dat", "X \t Y", [p_to_p_amp_avg, ref_noise_level])
a2, = nlfit(XMGRACE_INPUT_PATH / "noise-ptp.dat", "20*log10(A0*x + A1)", print_output=True, a0=1).values()

In [ ]:
m = Model(func_noise_linear)
params = m.make_params(a=a2, method='leastsq')
fit_noise_lin = m.fit(ref_noise_level,params, x=p_to_p_amp_avg)
print(fit_noise_lin.fit_report())
print(fit_noise_lin.params.pretty_print())

In [ ]:
fig, ax = plt.subplots()
p_to_p_amp_avg = np.array(p_to_p_amp_avg)
ax.errorbar(p_to_p_amp_avg , ref_noise_level, xerr=p_to_p_amp_err, fmt='+',capsize = 2, label=r"$U_{pop,SEN0487} (Me, \sigma)$", color=COLOR_BLUE)
ax.minorticks_on()
ax.tick_params(axis='both')
ax.set_xlabel('Peak-to-peak amplitude $(V)$')
ax.set_ylabel('Ref level $(dB)$')
ax.grid(True,'major','both')

ax.plot(p_to_p_amp_avg, fit_noise.best_fit, label = r'fit : $y=20 \times log_{10}(a \times x + b)$', color=COLOR_BLUE)    

ax.plot(p_to_p_amp_avg, fit_noise_lin.best_fit, '--', label = r'fit : $y=20 \times log_{10}(a \times x )$', color=COLOR_ORANGE)
ax.legend()
for ext in EXT_FILE:
    plt.savefig(DEST_PATH / Path(f"noise_fit.{ext}"))